# 한국어 형태소 분석

In [ ]:
df = pd.read_csv(f"{DATA_PATH}naver_review_train.csv")
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
df = df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149995 entries, 0 to 149994
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


## kiwi 라이브러리


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare() # 분석 준비

In [ ]:
# 첫번째 텍스트 담기
text = df["document"][0]
text

'아 더빙.. 진짜 짜증나네요 목소리'

In [ ]:
result = kiwi.analyze(text)
result

[([Token(form='아', tag='IC', start=0, len=1),
   Token(form='더빙', tag='NNG', start=2, len=2),
   Token(form='..', tag='SF', start=4, len=2),
   Token(form='진짜', tag='MAG', start=7, len=2),
   Token(form='짜증', tag='NNG', start=10, len=2),
   Token(form='나', tag='VV', start=12, len=1),
   Token(form='네요', tag='EF', start=13, len=2),
   Token(form='목소리', tag='NNG', start=16, len=3)],
  -70.44538879394531)]

In [ ]:
# 품사정보를 잘 가져와야함
result[0][0]

[Token(form='아', tag='IC', start=0, len=1),
 Token(form='더빙', tag='NNG', start=2, len=2),
 Token(form='..', tag='SF', start=4, len=2),
 Token(form='진짜', tag='MAG', start=7, len=2),
 Token(form='짜증', tag='NNG', start=10, len=2),
 Token(form='나', tag='VV', start=12, len=1),
 Token(form='네요', tag='EF', start=13, len=2),
 Token(form='목소리', tag='NNG', start=16, len=3)]

In [ ]:
lst = df["document"][:2].tolist()
lst

['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나']

In [ ]:
result = kiwi.analyze(lst) # 리스트를 넣으면 제너레이터 객체로 반환되어서 for문을 통해 꺼낼수있다.
result

In [ ]:
for tokens in result:
    print(tokens[0][0])

[Token(form='아', tag='IC', start=0, len=1), Token(form='더빙', tag='NNG', start=2, len=2), Token(form='..', tag='SF', start=4, len=2), Token(form='진짜', tag='MAG', start=7, len=2), Token(form='짜증', tag='NNG', start=10, len=2), Token(form='나', tag='VV', start=12, len=1), Token(form='네요', tag='EF', start=13, len=2), Token(form='목소리', tag='NNG', start=16, len=3)]
[Token(form='흠', tag='IC', start=0, len=1), Token(form='...', tag='SF', start=1, len=3), Token(form='포스터', tag='NNG', start=4, len=3), Token(form='보', tag='VV', start=7, len=1), Token(form='고', tag='EC', start=8, len=1), Token(form='초딩', tag='NNP', start=10, len=2), Token(form='영화', tag='NNG', start=12, len=2), Token(form='줄', tag='NNG', start=14, len=1), Token(form='....', tag='SF', start=15, len=4), Token(form='오버', tag='NNG', start=19, len=2), Token(form='연기', tag='NNG', start=21, len=2), Token(form='조차', tag='JX', start=23, len=2), Token(form='가볍', tag='VA-I', start=26, len=2), Token(form='지', tag='EC', start=28, len=1), Token(f

In [ ]:
# 명사,동사,형용사만 꺼내보자(전처리)
lst = df["document"].tolist()
gen = kiwi.analyze(lst)
train = [] # 토큰화된것을 담을 리스트

for tokens in tqdm(gen,total=len(lst)): # 데이터가 너무 많아 오래걸리므로 tqdm처리, 제너레이터는 길이를 알수 없으므로 퍼센트 표시불가, 그래서 total옵션으로 리스트의 길이를 준다.
    token = [token.form for token in tokens[0][0] if token.tag[0] in "NVJ"] # 명사,동사,형용사 tag에 해당된 토큰의 form만 골라줌
    train.append(token)

  0%|          | 0/149995 [00:00<?, ?it/s]

- 어휘집만들기

In [ ]:
vocab = build_vocab_from_iterator(train,specials=["<pad>","<unk>"])
vocab.set_default_index(vocab["<unk>"]) # 사전에 없는 문자가 오면 1로 바꾸겠다

In [ ]:
len(vocab)

42830

- 정수 인코딩

In [ ]:
train = [vocab(text) for text in train]
len(train)

149995

- 전체길이 맞추기

In [ ]:
max_len = max( len(lst) for lst in train)
max_len

72

In [ ]:
train = [ lst + [0] * (max_len - len(lst)) if len(lst) < max_len else lst for lst in train]
train = np.array(train)
train.shape

(149995, 72)

In [ ]:
target = df["label"].to_numpy().reshape(-1,1)
target.shape

(149995, 1)

# 데이터셋 만들기

In [ ]:
class Review_Ko_Dataset(torch.utils.data.Dataset):
    def __init__(self,x,y=None):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,idx):
        item = {}
        item["x"] = torch.LongTensor(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item

In [ ]:
dt = Review_Ko_Dataset(train,target)
dl = torch.utils.data.DataLoader(dt,batch_size=2,shuffle=False)
batch = next(iter(dl))
batch

{'x': tensor([[ 346,  138,   12,  469,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
         [ 321,    4,  434,    3,   90, 1078,   34,  533,  478,   31,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]),
 'y': t

In [ ]:
emb_layer = torch.nn.Embedding(len(vocab),4)
x = emb_layer(batch["x"])
x.shape

torch.Size([2, 72, 4])

In [ ]:
class Conv1dModel(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim): # 임베딩을 하기위한 vocab_size
        super().__init__()
        self.emb_layer = torch.nn.Embedding(vocab_size,embedding_dim)
        self.seq = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=embedding_dim,out_channels=embedding_dim*2,kernel_size=3), # 아웃풋 피쳐 2배
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(in_channels=embedding_dim*2,out_channels=embedding_dim*4,kernel_size=3), # 아웃풋 피쳐 2배를 줘서 입력을 똑같이 2배
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            torch.nn.AdaptiveAvgPool1d(1),
            torch.nn.Flatten(), # 배치차원을 제외시켜 평평하게 만듬
            torch.nn.Linear(embedding_dim*4,1) # 9번째 줄 아웃풋 4배 똑같이 하고, 1을 예측

        )

    def forward(self,x):
        x = self.emb_layer(x)
        x = x.permute(0,2,1)
        return self.seq(x)

In [ ]:
model = Conv1dModel(len(vocab),128)
model(batch["x"])

tensor([[-0.1847],
        [-0.1722]], grad_fn=<AddmmBackward0>)

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 # 미니배치별에 대한 loss들의 평균
    model.train()  # 학습모드
    for batch in dataloader:
        pred = model(batch["x"].to(device))
        #pred = pred.squeeze()
        loss = loss_fn(pred,batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward() # 손실에 대한 반영
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def test_loop(dataloader,model,loss_fn,device):
    epoch_loss = 0  # 미니배치별에 대한 loss들의 평균
    model.eval()    # 평가모드

    pred_list = [] # 내보내야할 예측값
    for batch in dataloader:
        pred = model(batch["x"].to(device))

        # y값이 None일경우
        if batch.get("y") is not None:
            loss = loss_fn(pred,batch["y"].to(device))
            epoch_loss += loss.item()

        pred = pred.to("cpu").numpy()
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list)
    return epoch_loss,pred

In [ ]:
n_splits = 5
vocab_size = len(vocab)
embedding_dim = 4
batch_size = 128
epochs = 100
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = False
reset_seeds(SEED)
best_score_list = []

for i,(tri,vai) in enumerate(cv.split(train)):
    model = Conv1dModel(vocab_size,embedding_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters())

    # dataset
    train_dt = Review_Ko_Dataset(train[tri],target[tri])
    valid_dt = Review_Ko_Dataset(train[vai],target[vai])

    # dataloader
    train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt,batch_size=batch_size,shuffle=False)
    
    #mae는 낮을수록 좋으니 5번의 갱신끝에 변화가 없으면 종료하려는 구문
    best_score = 0
    patience = 0 
    
    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss,pred = test_loop(valid_dl,model,loss_fn,device)

        pred =(pred > 0.5).astype(int)
        score = accuracy_score(target[vai],pred)
        #print(train_loss,valid_loss, accuracy_score(target[vai],pred))
        patience += 1

        if best_score < score:  # score가 더 작으면 patience 0으로 초기화
            patience = 0
            best_score = score  # 갱신
            torch.save(model.state_dict(),f"model_{i}.pth")

        if patience == 20:
            break
        print(train_loss,valid_loss, accuracy_score(target[vai],pred))
    print(f"Fold ({i}), BEST SCORE: {best_score}")
    best_score_list.append(best_score)
    if is_holdout:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

0.6780371372379473 0.6172362631939827 0.6193873129104304
0.5621073918301922 0.5238059164361751 0.717490583019434
0.4904487926695647 0.47997402003470885 0.7630254341811393
0.45394437822070455 0.45906163682328893 0.7770925697523251
0.4328412946099157 0.44689111062820924 0.7842261408713623
0.4177582188010978 0.44194363355636596 0.7942598086602887
0.40601406305202287 0.4338249649139161 0.7940264675489183
0.3956882313116273 0.4307473952465869 0.7983266108870296
0.3869853177622183 0.42631455304774835 0.7989599653321777
0.37902009582468693 0.42428028856186156 0.8014600486682889
0.3716321217734168 0.4238539944303797 0.8037267908930298
0.3649591619271968 0.421005820467117 0.8040934697823261
0.35876837595185235 0.423105207275837 0.8056601886729557
0.35300857794564416 0.4219820354213106 0.8058601953398447
0.34749510850924165 0.42290100379193085 0.8066602220074003
0.34217210701787903 0.42402023102374786 0.8069268975632521
0.3373270976994592 0.4260645957069194 0.8069935664522151
0.3325664738951716 

  0%|          | 0/100 [00:00<?, ?it/s]

0.660049145473346 0.600680237628044 0.6321877395913197
0.548500633093594 0.5137154400348664 0.7401580052668423
0.4793655665825679 0.4739080827286903 0.7675255841861396
0.44773197644300805 0.45559592082145367 0.7748591619720657
0.42817051378267407 0.4451003283896345 0.7880596019867329
0.4133482776216861 0.4379218172519765 0.7931264375479182
0.4011810219237037 0.43141189559977106 0.7954265142171406
0.3907872643043746 0.4278154473355476 0.7965598853295109
0.38180284198921627 0.4264170859722381 0.7981932731091036
0.3739049346176292 0.4245172378864694 0.8004600153338445
0.3667636267634343 0.42469187670565667 0.8020267342244741
0.36004333523735566 0.4259329885878462 0.8038267942264742
0.3538747223169565 0.4255271702370745 0.805193506450215
0.3482207149140108 0.4268924722012053 0.8043268108936965
0.3429870926209096 0.42801404329056436 0.8046601553385113
0.33762353733340816 0.4303817705905184 0.8048934964498816
0.33281032968241014 0.4323125127782213 0.8056268542284742
0.3282302879035346 0.4337

  0%|          | 0/100 [00:00<?, ?it/s]

0.661931751824137 0.5853006399692373 0.6736224540818028
0.5417602378358719 0.5111416163596701 0.736757891929731
0.4839368215374855 0.47593557834625244 0.7614587152905097
0.4523188286244488 0.45725276698457434 0.7739924664155472
0.43267804498611484 0.44825059505219156 0.7755925197506583
0.417983431964795 0.43812079810081644 0.7884262808760292
0.40581917839009624 0.43311328177756453 0.7948264942164739
0.39555429300265527 0.42868568123655115 0.7962265408846961
0.3865673257820388 0.42501588806192925 0.8003600120004
0.37828132676989284 0.4256739265107094 0.7979599319977333
0.37102898437458315 0.42296440373075767 0.8001266708890297
0.36422834904399765 0.42148403005397067 0.8063935464515484
0.3578912793223792 0.4214153639813687 0.8053268442281409
0.3519128775958822 0.4221713027421464 0.8067935597853262
0.3463771615836666 0.4219732127925183 0.8083269442314743
0.34136760299965774 0.4226800481689737 0.8104936831227708
0.3361184143149522 0.4255133777222735 0.8100603353445115
0.33151619881391525 0

  0%|          | 0/100 [00:00<?, ?it/s]

0.6488065889268033 0.5783101860513078 0.6642221407380245
0.5350972046730107 0.5051055118124536 0.7417913930464349
0.4781858872439561 0.47265363335609434 0.7646254875162506
0.449111562802085 0.45567990237093986 0.775225840861362
0.42976275296099403 0.4441998650418951 0.782192739757992
0.4153772175375587 0.43638069135077456 0.7900596686556218
0.4038948536173367 0.4306230051720396 0.7923930797693256
0.3940963412462267 0.42695927023887636 0.7942264742158072
0.3853167567902537 0.42411020398139954 0.7972265742191407
0.37795626667580373 0.42161510231647087 0.8005933531117704
0.3706810374472187 0.42023119000678366 0.8007266908896963
0.36402300155874506 0.41993753745200785 0.8033601120037335
0.3578969061787703 0.4196213705742613 0.8056268542284742
0.3519035647831746 0.41809393027995495 0.8037601253375113
0.3464923242706734 0.41998208482214744 0.8075269175639188
0.34132169744671026 0.42061116099357604 0.808093603120104
0.3362057320694171 0.42180835744167894 0.8067935597853262
0.331403623622999 0

  0%|          | 0/100 [00:00<?, ?it/s]

0.6645564055010709 0.5969024754585104 0.6411213707123571
0.5412457661588055 0.5147640658185837 0.7258241941398047
0.4818386092368982 0.47820434582994337 0.7590253008433614
0.45197514932292865 0.4597596606041523 0.7703256775225841
0.4330980322762593 0.44794109615873784 0.7786926230874363
0.4192755908282327 0.43970600366592405 0.7870595686522884
0.408269690584018 0.4342695733334156 0.7893263108770292
0.3986984160599678 0.43011256912921336 0.7923597453248442
0.39040914629059814 0.42690170775068564 0.7951931731057702
0.38280866050453327 0.42533962029091854 0.7961932064402146
0.37564500276722124 0.42287386592398296 0.7994266475549185
0.3690914199042168 0.4252082812025192 0.796626554218474
0.3628937246512248 0.42342258298650703 0.797593253108437
0.35720981157092907 0.4242999093329653 0.8007600253341778
0.3515597836517576 0.4231540653299778 0.8038601286709557
0.3463592202837533 0.4222300151561169 0.805193506450215
0.3414102145063598 0.42550272878180156 0.8053935131171039
0.3365306067409546 0.

In [ ]:
np.mean(best_score_list)

0.8081336044534819